In [10]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt

In [11]:
%%time
# Change this for validation with 10% from train
is_valid = False

path = 'input/'

def timeFeatures(df):
    # Make some new features with click_time column
    df['datetime'] = pd.to_datetime(df['click_time'])
    df['dow']      = df['datetime'].dt.dayofweek
    df["doy"]      = df["datetime"].dt.dayofyear
    #df["dteom"]    = df["datetime"].dt.daysinmonth - df["datetime"].dt.day
    df.drop(['click_time', 'datetime'], axis=1, inplace=True)
    return df

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

# Read the last lines because they are more impacting in training than the starting lines
train = pd.read_csv(path+"train.csv", skiprows=range(1,123903891), nrows=61000000, usecols=train_columns, dtype=dtypes)
test = pd.read_csv(path+"test_supplement.csv", usecols=test_columns, dtype=dtypes)

CPU times: user 2min 40s, sys: 12.5 s, total: 2min 53s
Wall time: 3min 51s


In [17]:
train.shape, test.shape

NameError: name 'test' is not defined

In [12]:
%%time
# Drop the IP and the columns from target
y = train['is_attributed']
train.drop(['is_attributed'], axis=1, inplace=True)

# Drop IP and ID from test rows
sub = pd.DataFrame()
#sub['click_id'] = test['click_id'].astype('int')
test.drop(['click_id'], axis=1, inplace=True)
gc.collect()

CPU times: user 2.8 s, sys: 1.88 s, total: 4.68 s
Wall time: 4.68 s


In [14]:
%%time
nrow_train = train.shape[0]
df_all = pd.concat([train, test])

del train, test
gc.collect()

# Count the number of clicks by ip
ip_count = df_all.groupby(['ip'])['channel'].count().reset_index()
ip_count.columns = ['ip', 'clicks_by_ip']
merge = pd.merge(df_all, ip_count, on='ip', how='left', sort=False)
df_all['clicks_by_ip'] = df_all['clicks_by_ip'].astype('uint16')
df_all.drop('ip', axis=1, inplace=True)

df_train = df_all[:nrow_train]
df_test = df_all[nrow_train:]

del merge
gc.collect()


NameError: name 'test' is not defined

In [ ]:
df_train = timeFeatures(df_train)
gc.collect()

In [15]:

# print('[{}] Start XGBoost Training'.format(time.time() - start_time))

# # Set the params(this params from Pranav kernel) for xgboost model
# params = {'eta': 0.3,
#           'tree_method': "hist",
#           'grow_policy': "lossguide",
#           'max_leaves': 1400,  
#           'max_depth': 0, 
#           'subsample': 0.9, 
#           'colsample_bytree': 0.7, 
#           'colsample_bylevel':0.7,
#           'min_child_weight':0,
#           'alpha':4,
#           'objective': 'binary:logistic', 
#           'scale_pos_weight':9,
#           'eval_metric': 'auc', 
#           'nthread':8,
#           'random_state': 99, 
#           'silent': True}
          

# if (is_valid == True):
#     # Get 10% of train dataset to use as validation
#     x1, x2, y1, y2 = train_test_split(train, y, test_size=0.1, random_state=99)
#     dtrain = xgb.DMatrix(x1, y1)
#     dvalid = xgb.DMatrix(x2, y2)
#     del x1, y1, x2, y2 
#     gc.collect()
#     watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
#     model = xgb.train(params, dtrain, 200, watchlist, maximize=True, early_stopping_rounds = 25, verbose_eval=5)
#     del dvalid
# else:
#     dtrain = xgb.DMatrix(train, y)
#     del train, y
#     gc.collect()
#     watchlist = [(dtrain, 'train')]
#     model = xgb.train(params, dtrain, 30, watchlist, maximize=True, verbose_eval=1)

# del dtrain
# gc.collect()

In [ ]:
%%time
model = lgb.LGBMClassifier(n_jobs=-1).fit(train, y)

In [ ]:
%%time
# Plot the feature importance from xgboost
# plot_importance(model)

# Load the test for predict 
test = pd.read_csv(path+"test.csv", usecols=test_columns, dtype=dtypes)
test = pd.merge(test, ip_count, on='ip', how='left', sort=False)
del ip_count
gc.collect()

sub['click_id'] = test['click_id'].astype('int')

test['clicks_by_ip'] = test['clicks_by_ip'].astype('uint16')
test = timeFeatures(test)
test.drop(['click_id', 'ip'], axis=1, inplace=True)
y_pred = model.predict(test)
del test
gc.collect()

# Save the predictions
sub['is_attributed'] = y_pred
sub.to_csv('model_lgb.csv', float_format='%.8f', index=False)

!kaggle competitions submit -c talkingdata-adtracking-fraud-detection -f model_lgb.csv -m "Message"

In [ ]:
model = lgb.